<a href="https://colab.research.google.com/github/BasitHussain5/Bangala-Text-NLP-Project/blob/main/EDA%2CPreprocessing_On_Bangla_Text_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bangla-stopwords:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F806606%2F1382412%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241007%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241007T051649Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3327cd837c5411c4684acc807d27c348ca14838d222d7f0aa3c2e1db5b66ac755e5f4f4949344b31df421105eb4fc50aee14cf725df4c26d5ffbba8b5473fd80a0e19854e24016cf185c483ddf30e0af32f95a2fb0ac154bb24083d5e0b4d03521d33c675c15bae1960022e97a660b5ad8228910133e5f7ed320c6453c5b6fe07f50c5a146bf58950447a015d850436194cf532079af862d42acaaa71647e5809180b95a2b390309c2858038487ece393d1dceec5de8d0bca579fb479d3545c071f633b73e931126b0598e0b6e2deadb3da8698178c1733cd1e6cd84b14721a54173c683f30fa50fb0299be683be136bffb4b8495c556ae62d46212df8215aff,siyam-rupali-text:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F989926%2F1671368%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241007%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241007T051649Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D163c75c8f0e9f141e3e8cd9458c4ed9d35ed1f143fe241ec569f68ed6480e6b71d52dc49c2761666feac74cd03a0a7e2626290eafce06106e246fe11bee7e18cf143892990fb9e5a1432ebb0e13f34c2266e19ec889eee0ceedda6656927ccac54191699e977263cd1609839fec7d0ce07421d754956a100bde38fb7a7ed410b29c5e9ca19bf0a27c363aa6cb2324681fb0164829f1dc604e55d13dfc15cf222334749194b71293bf6b779dc70fb8cc47db2b3ad816cf4aeaa2708758f908206d5845ff38eebbfb646f2e528e614cc2ba8ae19439deab929d11fffd97f933a35fdb55ed33cb0ef3629d85a3fddfe4139b70050b4188a56d2a16742f7ccc055ce,textcategorizationbangla:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1755595%2F2866856%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241007%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241007T051649Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D644187bb06193811c73fe6b679ad78369a52f440d0a687345e5c9f76a1b90baf4e32084c1d92dbd6dba70cb470d2dff06fd3dcf4187d6701f44af149b6928e90f2ccea016916c8a7ef8de3b3b0bf2c452eff6f7f3e5100754c6871ec71ab58fb39632426a03541bf90384be0174d39df4be59f478a158b29501a4cb0c07111f24496640de678741b33828be6d9c3e0fe48da84d22a7df1d7290ec079865a399a40659a9156221b14b5ffcb501b87744e3a8478268f369eb890d851119703e2aaa3e9946eec6231865ed51332accac22375e2b7593ed947efb97eef022195568d5018f6e5ac45dfedac09e898e73bde9b11e935f0327cc4f012d2c303e1b5eda0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# **Updating/Installing Libraries**

In [ ]:
#!pip install keras
#!pip install --upgrade tensorflow
#!pip install --upgrade tensorflow-gpu
!pip install xlrd==1.2.0
!pip install openpyxl

# **Importing Libraries**

In [ ]:
import pandas as pd
from pandas import read_excel
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode
import os
from time import time
from collections import Counter

from sklearn import metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import LSTM,Dense,Dropout,Activation,Embedding,Flatten,Bidirectional,MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.python.keras.models import Sequential

from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras import regularizers

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


import h5py
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score


from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

# **Loading Dataset**

In [ ]:
df_train=pd.read_csv("/kaggle/input/textcategorizationbangla/train_data_stemmer.csv")
df_test=pd.read_csv("/kaggle/input/textcategorizationbangla/test_data_stemmer.csv")


In [ ]:
display(df_train)
display(df_test)

# **TrainSet Description**

In [ ]:
print("First rows of TrainSet....")
df_train.head()

In [ ]:
print("Description of TrainSet...")
df_train.describe()

In [ ]:
print("Dataframe Information on TrainSet...")
df_train.info()

In [ ]:
print("Categories of Trainset:")
print(df_train.category.unique())
category_list=df_train.category.unique()

# **Testset Description**

In [ ]:
print("First rows of TrainSet....")
df_test.head()

In [ ]:
print("Description of TrainSe....")
df_test.describe()

In [ ]:
print("Information of TrainSet dataframe....")
df_test.info()

In [ ]:
print("Categories:")
print(df_train.category.unique())
category_list=df_train.category.unique()

# **Removing Null Values**

In [ ]:
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

# **Removing Low Length Data**

In [ ]:
#counting text length
df_train['count'] = df_train['cleanText'].str.split().str.len()
df_test['count'] = df_test['cleanText'].str.split().str.len()
# Remove the text with words less than 5
df_train= df_train.loc[df_train['count']>5]
df_test= df_test.loc[df_test['count']>5]

# **Preprocessing/Cleaning**
Though the dataset is said to have been cleaned,but i have applied it anyway to remove Unecessary charachters,Emojis,Punctuations with the help of Regex.

In [ ]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

In [ ]:
#df_train['cleanText'] = df_train.cleanText.apply(lambda x: preprocessing(str(x)))
#df_test['cleanText'] = df_test.cleanText.apply(lambda x: preprocessing(str(x)))

# **Stopwords Removal**

This is a stopwords list containing about 700+ bangla stopwords. I have manually collected this stopwords.

In [ ]:
data1 =pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx')
stop = data1['words'].tolist()

In [ ]:
display(data1)

In [ ]:
def stopwordRemoval(text):
    x=str(text)
    l=x.split()

    stm=[elem for elem in l if elem not in stop]

    out=' '.join(stm)

    return str(out)

In [ ]:
#df_train['cleanText'] = df_train.cleanText.apply(lambda x: stopwordRemoval(str(x)))
#df_test['cleanText'] = df_test.cleanText.apply(lambda x: stopwordRemoval(str(x)))

# **Stemming**

In [ ]:
#make sure to turn on internet on your kernel
#importing stemmer
!pip install bangla-stemmer
from bangla_stemmer.stemmer import stemmer
## stemmer function
def stem_text (x):
  stmr = stemmer.BanglaStemmer()
  words=x.split(' ')
  stm = stmr.stem(words)
  words=(' ').join(stm)
  return words

In [ ]:
#df_train['cleanText']=df_train['cleanText'].apply(stem_text)
#df_test['cleanText']=df_test['cleanText'].apply(stem_text)

I have used a Bangla Stemmer for the dataset. But through experimentation,i have seen than stemming doesn't have that much effect on our accuracy measures. So i have commented out the corresponding code.

# **Category Wise Data Distribution**

In [ ]:
sns.countplot(df_train['category']);

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.countplot(df_test['category']);

In both train set and test set,the data have no imbalance problem. We can see that every class have sufficient amount of data.

# **Count of Texts in Each Category**

In [ ]:
print("IN TRAIN SET...")
temp1 = df_train.groupby('category').count()['cleanText'].reset_index().sort_values(by='cleanText',ascending=False)
temp1.style.background_gradient(cmap='Purples')

In [ ]:
print("In Test Set...")
temp2 = df_test.groupby('category').count()['cleanText'].reset_index().sort_values(by='cleanText',ascending=False)
temp2.style.background_gradient(cmap='Purples')

# **Funnel Chart of Data Distribution**

In [ ]:
from plotly import graph_objs as go
print("On Train Set....")
fig = go.Figure(go.Funnelarea(
    text =temp1.category,
    values = temp1.cleanText,
    title = {"position": "top center", "text": "Funnel-Chart of Category Distribution on Train Set"}
    ))
fig.show()

In [ ]:
print("On Test Set...")
fig = go.Figure(go.Funnelarea(
    text =temp2.category,
    values = temp2.cleanText,
    title = {"position": "top center", "text": "Funnel-Chart of Category Distribution on Test Set"}
    ))
fig.show()

# **Count Based Top Words in Each Category on Train Set**

In [ ]:
category_list=category_list.tolist()
for i in category_list:
    temp=df_train.loc[df_train['category'] == str(i)]
    #display(temp)
    temp['temp_list'] = temp['cleanText'].apply(lambda x:str(x).split())
    top = Counter([item for sublist in temp['temp_list'] for item in sublist])
    temp = pd.DataFrame(top.most_common(20))
    temp.columns = ['Common_words','count']
    temp.style.background_gradient(cmap='Blues')
    temp = temp.style.set_caption('Top 20 Words In '+ str(i)+" Category")
    display(temp)


# **Top 20 Words in TrainSet Based on Count**

In [ ]:

df_train['temp_list'] = df_train['cleanText'].apply(lambda x:str(x).split())
top = Counter([item for sublist in df_train['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

# **Top 20 Words in TestSet Based on Count**

In [ ]:

df_test['temp_list'] = df_test['cleanText'].apply(lambda x:str(x).split())
top = Counter([item for sublist in df_test['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

# **Histogram Analysis Based on Text Length**

In [ ]:
fig = go.Figure(layout=dict(title=dict(text="Text Length Histogram of Trainset")))
fig.add_trace(go.Histogram(x=df_train['count']))
fig.show()

In [ ]:
fig = go.Figure(layout=dict(title=dict(text="Text Length Histogram of Testset")))
fig.add_trace(go.Histogram(x=df_test['count']))
fig.show()

From the histogram,we can say that abobut 97% of the text lenght are between 1-600. This information is valuable for model creation and text lenght selection.

# **N gram Analysis**

In [ ]:
df_train=pd.read_csv("/kaggle/input/textcategorizationbangla/train_data_stemmer.csv")
df_test=pd.read_csv("/kaggle/input/textcategorizationbangla/test_data_stemmer.csv")


In [ ]:
# function to create top 20 n-grams
def get_ngrams(data,n):
    all_words = []
    for i in range(len(data)):
        temp = data["cleanText"][i].split()
        for word in temp:
            all_words.append(word)

    tokenized = all_words
    esBigrams = ngrams(tokenized, n)

    esBigram_wordlist = nltk.FreqDist(esBigrams)
    top20 = esBigram_wordlist.most_common(20)
    top20 = dict(top20)
    df_ngrams = pd.DataFrame(sorted(top20.items(), key=lambda x: x[1])[::-1])
    df_ngrams.columns = ['Ngram','count']
    return df_ngrams


# function to visualize the top 20 n-grams
def show(train):
    display(train.head(20))

# **Top 20 Unigram Count Based On Category**

In [ ]:
for i in category_list:
    temp=df_train.loc[df_train['category'] == str(i)]
    #display(temp)
    temp['temp_list'] = temp['cleanText'].apply(lambda x:str(x).split())
    temp.reset_index(drop=True, inplace=True)
    train_unigrams = get_ngrams(temp,1)
    print("\t\t\t====== Unigrams of "+str(i)+"======")
    show(train_unigrams)

# **Top 20 Bigram Count Based On Category**

In [ ]:
for i in category_list:
    temp=df_train.loc[df_train['category'] == str(i)]
    #display(temp)
    temp['temp_list'] = temp['cleanText'].apply(lambda x:str(x).split())
    temp.reset_index(drop=True, inplace=True)
    train_bigrams = get_ngrams(temp,2)
    print("\t\t\t====== Bigrams of "+str(i)+" ======")
    show(train_bigrams)

# **Top 20 Trigram Count Based On Category**

In [ ]:
for i in category_list:
    temp=df_train.loc[df_train['category'] == str(i)]
    #display(temp)
    temp['temp_list'] = temp['cleanText'].apply(lambda x:str(x).split())
    temp.reset_index(drop=True, inplace=True)
    train_trigrams = get_ngrams(temp,3)
    print("\t\t\t====== Trigrams of "+str(i)+" ======")
    show(train_trigrams)

# **Word Cloud Based On Category**

In [ ]:
# Importing wordcloud for plotting word clouds and textwrap for wrapping longer text
from wordcloud import WordCloud
from textwrap import wrap

import matplotlib.pyplot as plt
from matplotlib import font_manager

# Function for generating word clouds
def generate_wordcloud(data,title):
  data = [tuple(x) for x in data.values]
  wc = WordCloud(font_path="/kaggle/input/siyam-rupali-text/Siyamrupali.ttf",width=1080, height=720, max_words=150,colormap="Dark2").generate_from_frequencies(dict(data))
  plt.figure(figsize=(10,8))
  plt.imshow(wc, interpolation='bilinear')
  plt.axis("off")
  plt.title('\n'.join(wrap("Word Cloud of "+title,60)),fontsize=13)
  plt.show()

In [ ]:

for i in category_list:
    temp=df_train.loc[df_train['category'] == str(i)]
    #display(temp)
    temp['temp_list'] = temp['cleanText'].apply(lambda x:str(x).split())
    top = Counter([item for sublist in temp['temp_list'] for item in sublist])
    temp = pd.DataFrame(top.most_common(500000))
    temp.columns = ['Common_words','count']
    generate_wordcloud(temp,str(i))


# **Conclusion Of EDA on banglaMCT7 Dataset**
* **Both TrainSet and TrainSet are similar in size.**
* **TraintSet and TestSet top word counts are similar,so we can say that TrainSet and TestSet are originated from same distribution of Data.**
* **The dataset don't have any data imbalance problem,every category have sufficient amount of data.**
* **Histogram analaysis shows that most of the text are of lenght 1-600.**
* **Category wise Ngram shows that the category wised created ngrams are relavent to that category.**
* **Category wise Word Cloud also shows result relavent to the category**